In [1]:
import os
import pandas as pd
import numpy as np
from env import get_connection
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import acquire as ak
import prepare as prep

# 4. In a jupyter notebook, classification_exercises.ipynb, use a python module (pydata or seaborn datasets) containing datasets as a source from the iris data. Create a pandas dataframe, df_iris, from this data.

In [2]:
db_url = get_connection('iris_db')
query_one = '''

SELECT *
FROM measurements

'''

query_two = '''

SELECT *
FROM species

'''

In [3]:
iris_m = pd.read_sql(query_one, db_url)
iris_s = pd.read_sql(query_two, db_url)

# print the first 3 rows

In [4]:
# the iris measurements db as a df
iris_m.head(3)

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_id
0,1,5.1,3.5,1.4,0.2,1
1,2,4.9,3.0,1.4,0.2,1
2,3,4.7,3.2,1.3,0.2,1


In [5]:
# the iris species db as df
iris_s.head()

,species_id,species_name
0,1,setosa
1,2,versicolor
2,3,virginica


# print the number of rows and columns (shape)

In [6]:
iris_m.shape

(150, 6)

# print the column names
# print the data type of each column

In [7]:
iris_m.columns.tolist()

['measurement_id',
 'sepal_length',
 'sepal_width',
 'petal_length',
 'petal_width',
 'species_id']

In [8]:
iris_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   measurement_id  150 non-null    int64  
 1   sepal_length    150 non-null    float64
 2   sepal_width     150 non-null    float64
 3   petal_length    150 non-null    float64
 4   petal_width     150 non-null    float64
 5   species_id      150 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 7.2 KB


# print the summary statistics for each of the numeric variables

In [9]:
iris_m.describe()

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_id
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.057333,3.758000,1.199333,2.000000
std,43.445368,0.828066,0.435866,1.765298,0.762238,0.819232
min,1.000000,4.300000,2.000000,1.000000,0.100000,1.000000
25%,38.250000,5.100000,2.800000,1.600000,0.300000,1.000000
50%,75.500000,5.800000,3.000000,4.350000,1.300000,2.000000
75%,112.750000,6.400000,3.300000,5.100000,1.800000,3.000000
max,150.000000,7.900000,4.400000,6.900000,2.500000,3.000000


# 5. Read the data from this google sheet into a dataframe, df_google.

In [10]:
train = pd.read_excel('/Users/edwige/downloads/train.xlsx')

# print the first 3 rows

In [11]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


# print the number of rows and columns

In [12]:
train.shape

(891, 12)

# print the column names

# print the data type of each column

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [14]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

# print the summary statistics for each of the numeric variables

In [15]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# print the unique values for each of your categorical variables

In [16]:
# UNIQUE VALUES For all catagorical classes 
print(train.Survived.unique())
print(train.Pclass.unique())
print(train.Sex.unique())
print(train.SibSp.unique())
print(train.Parch.unique())
print(train.Embarked.unique())

[0 1]
[3 1 2]
['male' 'female']
[1 0 3 4 2 5 8]
[0 1 2 5 3 4 6]
['S' 'C' 'Q' nan]


In [17]:
train.dtypes == 'int64'

PassengerId     True
Survived        True
Pclass          True
Name           False
Sex            False
Age            False
SibSp           True
Parch           True
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool

In [18]:
# setting up variable to iterate over index if I have times. 
##Refactor below awnser into something cleaner and more programatic.

cata = train.nunique()
cata

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [19]:
# I would need to itterate value counts to ittereate a list to print out all teh unique value contained.

In [20]:
# This is a more programatic way Need to prse it
# # Dictionary comprehension to get desired result
# result = {col: df[col].unique().tolist() for col in df.columns if df[col].nunique() < 8}

# 6. Download the previous exercise's file into an excel (File → Download → Microsoft Excel). Read the downloaded file into a dataframe named df_excel.


In [21]:
df_excel = pd.read_excel('/Users/edwige/downloads/train.xlsx')

# assign the first 100 rows to a new dataframe, df_excel_sample

In [22]:
df_excel_sample = df_excel.head(100)

# print the number of rows of your original dataframe

In [23]:
len(df_excel)

891

In [24]:
df_excel.shape[0]

891

# print the first 5 column names

In [25]:
df_excel.columns[:5].tolist()

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex']

# print the column names that have a data type of object

In [26]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [27]:
objex = df_excel.select_dtypes(include='object').columns.tolist()
objex

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

# compute the range for each of the numeric variables.

In [28]:
exc_des = df_excel.describe()

In [29]:
# This is the excel range
exc_des.loc['max'] - exc_des.loc['min']

PassengerId    890.0000
Survived         1.0000
Pclass           2.0000
Age             79.5800
SibSp            8.0000
Parch            6.0000
Fare           512.3292
dtype: float64

In [30]:
def get_titanic_data():
    filename = 'titanic_data.csv'
    
    if os.path.isfile(filename):
        return pd.read_csv(filename)

    else:
        url = get_connection('titanic_db')
        query = '''
        
        SELECT *
        FROM passengers
        ;
        '''

        titanic_data = pd.read_sql(query,url)
        titanic_data.to_csv(filename,index=0)
        return titanic_data

In [31]:
get_titanic_data()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,0,0,13.0000,S,Second,NaN,Southampton,1
887,887,1,1,female,19.0,0,0,30.0000,S,First,B,Southampton,1
888,888,0,3,female,NaN,1,2,23.4500,S,Third,NaN,Southampton,0
889,889,1,1,male,26.0,0,0,30.0000,C,First,C,Cherbourg,1


# Using the Iris Data:

1. Use the function defined in acquire.py to load the iris data.

In [32]:
import acquire as ak
import prepare as prep

In [33]:
ak.get_iris_data()

,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_id,species_id.1,species_name
0,1,5.1,3.5,1.4,0.2,1,1,setosa
1,2,4.9,3.0,1.4,0.2,1,1,setosa
2,3,4.7,3.2,1.3,0.2,1,1,setosa
3,4,4.6,3.1,1.5,0.2,1,1,setosa
4,5,5.0,3.6,1.4,0.2,1,1,setosa
...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,3,3,virginica
146,147,6.3,2.5,5.0,1.9,3,3,virginica
147,148,6.5,3.0,5.2,2.0,3,3,virginica
148,149,6.2,3.4,5.4,2.3,3,3,virginica


In [34]:
iris = ak.get_iris_data()

2. Drop the species_id and measurement_id columns.

In [35]:
iris = iris.drop(columns=['species_id', 'measurement_id'])

In [36]:
iris = iris.drop(columns=['species_id.1'])

3. Rename the species_name column to just species.

In [37]:
iris = iris.rename(columns ={'species_name':'species'})

In [38]:
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


4. Create dummy variables of the species name and concatenate onto the iris dataframe. (This is for practice, we don't always have to encode the target, but if we used species as a feature, we would need to encode it).

# skipping Dummy variables

5. Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied.

In [39]:
def prep_iris(i):
    a = i
    
    a = a.drop(columns=['species_id','measurement_id'])
    a = a.drop(columns=['species_id.1'])
    a = a.rename(columns ={'species_name':'species'})
    return a 
    
    

# Using the Titanic dataset

1. Use the function defined in acquire.py to load the Titanic data.

In [40]:
titan = ak.get_titanic_data()

2. Drop any unnecessary, unhelpful, or duplicated columns. This could mean dropping foreign key columns but keeping the corresponding string values, for example.

-- Dropping passenger_id because it doesn't hold value. More serves as an index and not predictive of anything

-- Dropping embraked beacsue it contains redudntant information. 

-- Drop deck becasue it contains many null values

-- Droping class because it contains redundant inforamtion to class.

-- Will need to clean up embark town and age becasue of null values

In [41]:
titanic = titan.drop(columns=['passenger_id','embarked', 'deck', 'class'])

In [42]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,Southampton,0
1,1,1,female,38.0,1,0,71.2833,Cherbourg,0
2,1,3,female,26.0,0,0,7.9250,Southampton,1
3,1,1,female,35.0,1,0,53.1000,Southampton,0
4,0,3,male,35.0,0,0,8.0500,Southampton,1
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,Southampton,1
887,1,1,female,19.0,0,0,30.0000,Southampton,1
888,0,3,female,NaN,1,2,23.4500,Southampton,0
889,1,1,male,26.0,0,0,30.0000,Cherbourg,1


3. Encode the categorical columns. Create dummy variables of the categorical columns and concatenate them onto the dataframe.

-- Skipping Dummy variables

4. Create a function named prep_titanic that accepts the raw titanic data, and returns the data with the transformations above applied.

In [43]:
def prep_titanic(i):
    a = i
    a = a.drop(columns=['passenger_id','embarked', 'deck', 'class'])
 
    return a 
    

In [ ]:
# .dropna()
## This method drops nulls

# Using the Telco dataset

1. Use the function defined in acquire.py to load the Telco data.

In [44]:
telco = ak.get_telco_data()

2. Drop any unnecessary, unhelpful, or duplicated columns. This could mean dropping foreign key columns but keeping the corresponding string values, for example.

In [45]:
telco.head(10)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,2,65.60,593.3,No,2,One year,1,DSL,2,Mailed check
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,2,59.90,542.4,No,1,Month-to-month,1,DSL,2,Mailed check
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,1,73.90,280.85,Yes,1,Month-to-month,2,Fiber optic,1,Electronic check
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,No,...,1,98.00,1237.85,Yes,1,Month-to-month,2,Fiber optic,1,Electronic check
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,No,...,2,83.90,267.4,Yes,1,Month-to-month,2,Fiber optic,2,Mailed check
5,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,4,69.40,571.45,No,1,Month-to-month,1,DSL,4,Credit card (automatic)
6,0013-SMEOE,Female,1,Yes,No,71,Yes,No,2,Yes,...,3,109.70,7904.25,No,3,Two year,2,Fiber optic,3,Bank transfer (automatic)
7,0014-BMAQU,Male,0,Yes,No,63,Yes,Yes,2,Yes,...,4,84.65,5377.8,No,3,Two year,2,Fiber optic,4,Credit card (automatic)
8,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,1,48.20,340.35,No,1,Month-to-month,1,DSL,1,Electronic check
9,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,2,90.45,5957.9,No,3,Two year,1,DSL,2,Mailed check


In [46]:
teltel = telco.drop(columns=['contract_type_id.1', 'contract_type_id', 'internet_service_type_id.1' , 'payment_type_id.1', 'payment_type_id', 'internet_service_type_id', 'customer_id', 'internet_service_type'])

3. Encode the categorical columns. Create dummy variables of the categorical columns and concatenate them onto the dataframe.

# Skipping Dummy variables

4. Create a function named prep_telco that accepts the raw telco data, and returns the data with the transformations above applied

In [47]:
def prep_telco(i):
    a = i
    a = a.drop(columns=['contract_type_id.1', 'contract_type_id', 'internet_service_type_id.1' , 'payment_type_id.1', 'payment_type_id', 'internet_service_type_id', 'customer_id', 'internet_service_type'])
 
    return a 

# Split your data

1. Write a function to split your data into train, test and validate datasets. Add this function to prepare.py.

In [48]:
def train_val_test(df, strat, seed = 55):
    train, val_test = train_test_split(df, train_size = 0.7,
                                      random_state = seed,
                                      stratify = df[strat])
    val, test = train_test_split(val_test, train_size=0.5,
                                random_state = seed,
                                stratify = val_test[strat])
    return train, val, test

2. Run the function in your notebook on the Iris dataset, returning 3 datasets, train_iris, validate_iris and test_iris.

In [57]:
p_iris = prep.prep_iris(ak.get_iris_data())

In [58]:
t_iris , val_iris , test_iris = prep.train_val_test(p_iris,'species',seed=55)

3. Run the function on the Titanic dataset, returning 3 datasets, train_titanic, validate_titanic and test_titanic.

In [49]:
p_titan = prep.prep_titanic(ak.get_titanic_data())

In [52]:
t_titan , val_titan , test_titan = prep.train_val_test(p_titan,'sex',seed=55)

4. Run the function on the Telco dataset, returning 3 datasets, train_telco, validate_telco and test_telco.

In [54]:
p_telco =  prep.prep_telco(ak.get_telco_data()) 

In [55]:
t_telco , val_telco , test_telco = prep.train_val_test(p_telco,'gender',seed=55)